In [1]:
import nibabel as nib
import os
from os import path
import numpy as np

In [2]:
subnum = 'sub-CC0007'
editp = '/Users/zeynepenkavi/Downloads/ConteQC/pass0_edits'
uneditp = '/Users/zeynepenkavi/Downloads/ConteQC/pass0'

In [6]:
vols = [i for i in os.listdir(path.join(editp, subnum)) if i.endswith('.mgz')]

In [8]:
for vol in vols:

    edit_img = nib.load(path.join(editp,'%s/%s')%(subnum, vol))
    edit_data = edit_img.get_fdata()
    unedit_img = nib.load(path.join(uneditp,'%s/mri/%s')%(subnum, vol))
    unedit_data = unedit_img.get_fdata()
    diff_data = unedit_data - edit_data


    diff_img = nib.MGHImage(diff_data.astype(np.int32), edit_img.affine)
    nib.save(diff_img, os.path.join(editp, subnum,'diff_%s')%(vol))